In [1]:
pip install psycopg2-binary


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 991.0 kB/s eta 0:00:02
   ---- ----------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.2 MB 2.4 MB/s eta 0:00:01
   ---------------- ----------------------- 0.5/1.2 MB 2.8 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 2.5 MB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.2 MB 2.8 MB/s eta 0:00:01
   ----------------------------------- ---- 1.0/1.2 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.2 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, Float, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import re
from sklearn.impute import KNNImputer
from sqlalchemy import text
from datetime import datetime, timedelta

# Define the database model using SQLAlchemy's Declarative System
Base = declarative_base()

# Define a class for the OptimusData table
class OptimusData(Base):
    __tablename__ = 'optimus_data'
    id = Column(Integer, primary_key=True, autoincrement=True)
    # Define columns based on the structure of your Excel file or data source
    TRANSPORTATION_COST = Column(Float)
    QTY = Column(Integer)
    SKU_REVENUE = Column(Float)
    Total_Pallets = Column(Integer)
    Total_Packages = Column(Integer)
    Total_MPF = Column(Float)
    # Add additional fields as necessary based on your data structure

# Define additional models for storing KPIs (Key Performance Indicators)
class MonthlyKPIs(Base):
    __tablename__ = 'monthly_kpis'
    id = Column(Integer, primary_key=True, autoincrement=True)
    Year = Column(Integer)
    Month = Column(String)
    Total_Units = Column(Integer)
    Total_Pallets = Column(Integer)
    Total_Packages = Column(Integer)
    Total_MPF = Column(Float)

class TotalMPFPerTransportType(Base):
    __tablename__ = 'total_mpf_per_transport_type'
    id = Column(Integer, primary_key=True, autoincrement=True)
    TRANSPORT_TYPE = Column(String)
    Total_MPF = Column(Float)

class TotalRevenuePerCategory(Base):
    __tablename__ = 'total_revenue_per_category'
    id = Column(Integer, primary_key=True, autoincrement=True)
    CATEGORY = Column(String)
    SKU_REVENUE = Column(Float)

class TotalTransportCostPerConsignee(Base):
    __tablename__ = 'total_transport_cost_per_consignee'
    id = Column(Integer, primary_key=True, autoincrement=True)
    CONSIGNEE = Column(String)
    TRANSPORTATION_COST = Column(Float)

# Define a class for the DateDimension table to store date-related information
class DateDimension(Base):
    __tablename__ = 'date_dimension'
    date = Column(DateTime, primary_key=True)
    year = Column(Integer)
    month = Column(Integer)
    day = Column(Integer)
    week = Column(Integer)
    quarter = Column(Integer)
    day_of_week = Column(Integer)
    month_name = Column(String)
    day_name = Column(String)

# Initialize the database connection and create tables if they don't exist
engine = create_engine('sqlite:///optimus_data.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Function to populate the DateDimension table with dates between start_date and end_date
def populate_date_dimension(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        date_dim = DateDimension(
            date=current_date,
            year=current_date.year,
            month=current_date.month,
            day=current_date.day,
            week=current_date.isocalendar()[1],
            quarter=(current_date.month - 1) // 3 + 1,
            day_of_week=current_date.weekday(),
            month_name=current_date.strftime('%B'),
            day_name=current_date.strftime('%A'),
        )
        session.add(date_dim)
        current_date += timedelta(days=1)
    session.commit()

# Example usage of the function to populate the DateDimension table
populate_date_dimension(datetime(2010, 1, 1), datetime(2030, 12, 31))

# Function to generate a data quality report for a given DataFrame
def generate_data_quality_report(df, output_file_path):
    # Calculate the total number of rows in the DataFrame
    total_rows = {'Total Rows': [len(df)]}
    df_total_rows = pd.DataFrame(total_rows)
    
    # Calculate missing values count before any data imputation
    missing_values_before = df.isnull().sum()
    
    # Generate a basic data summary for each column in the DataFrame
    data_summary = df.describe(include='all')
    
    # Save the generated reports to an Excel file at the specified path
    with pd.ExcelWriter(output_file_path) as writer:
        df_total_rows.to_excel(writer, sheet_name='Total Rows', index=False)
        missing_values_before.to_frame(name='Missing Values Before').to_excel(writer, sheet_name='Missing Values Before')
        data_summary.to_excel(writer, sheet_name='Data Summary')
        
    print(f"Data quality report saved to {output_file_path}")

# Function to process Optimus data, generate KPIs, and insert data into the database
def process_optimus_data(file_path, quality_report_path):
    # Nested function to parse packing details from a row of data
    def parse_packing_details(row):
        pallets, packages = 0, 0
        # Iterate through specific columns containing packing details and parse them
        for col in ['PACKING_DETAILS_01', 'PACKING_DETAILS_02', 'PACKING_DETAILS_03', 'PACKING_DETAILS_04']:
            if pd.notna(row[col]):
                details = row[col].split(';')
                if 'PALLET' in details:
                    pallets += int(details[0])
                elif 'PACKAGE' in details:
                    packages += int(details[0])
        return pallets, packages

    # Function to calculate the MPF based on transportation cost
    def calculate_mpf(transportation_cost):
        if transportation_cost < 2500:
            return 2.18
        mpf = transportation_cost * 0.003464
        return max(26.22, min(mpf, 508.70))

    # Read data from the specified Excel file
    df = pd.read_excel(file_path, sheet_name='DATA')
    
    # Generate a Data Quality Report before performing any imputation
    generate_data_quality_report(df, quality_report_path)
    
    # Debugging: Print the shape of the DataFrame before imputation
    print("Shape of the DataFrame before imputation:", df.shape)
    
    # Identify numerical columns for imputation
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    imputer = KNNImputer()
    imputed_data = imputer.fit_transform(df[numerical_cols])

    # Debugging: Print information about imputed data
    print("Numerical columns selected for imputation:", numerical_cols)
    print("Shape of imputed data:", imputed_data.shape)
    
    # Update the original DataFrame with the imputed numerical data
    df[numerical_cols] = pd.DataFrame(imputed_data, columns=numerical_cols, index=df.index)

    # Fill missing values in non-numerical columns with 'Unknown'
    non_numerical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
    for col in non_numerical_cols:
        df[col] = df[col].fillna('Unknown')

    # Apply parsing function to each row to extract packing details
    df[['Total_Pallets', 'Total_Packages']] = df.apply(parse_packing_details, axis=1, result_type='expand')
    # Apply function to calculate MPF based on transportation cost
    df['Total_MPF'] = df['TRANSPORTATION_COST'].apply(calculate_mpf)

    # Convert pickup dates to datetime format and extract month and year
    df['PICKUP_DATE'] = pd.to_datetime(df['PICKUP_DATE'].apply(lambda x: x.split(' ')[0]), errors='coerce')
    df['Month'] = df['PICKUP_DATE'].dt.month
    df['Year'] = df['PICKUP_DATE'].dt.year

    # Aggregate data to calculate monthly totals for various KPIs
    monthly_totals = df.groupby(['Year', 'Month']).agg(
        Total_Units=('QTY', 'sum'),
        Total_Pallets=('Total_Pallets', 'sum'),
        Total_Packages=('Total_Packages', 'sum'),
        Total_MPF=('Total_MPF', 'sum')
    ).reset_index()

    # Calculate baseline values for comparison
    baseline = monthly_totals[monthly_totals['Month'].isin([1, 2])].groupby('Year').mean().reset_index()
    baseline['Month'] = 'Baseline'
    monthly_kpis = pd.concat([baseline, monthly_totals])

    # Aggregate data for other KPIs based on different groupings
    total_mpf_per_transport_type = df.groupby('TRANSPORT_TYPE')['Total_MPF'].sum().reset_index()
    total_revenue_per_category = df.groupby('CATEGORY')['SKU_REVENUE'].sum().reset_index()
    total_transport_cost_per_consignee = df.groupby('CONSIGNEE')['TRANSPORTATION_COST'].sum().reset_index()

    # Insert KPIs into the database
    insert_kpis_into_db(session, monthly_kpis, MonthlyKPIs, [
        'Year', 'Month', 'Total_Units', 'Total_Pallets', 'Total_Packages', 'Total_MPF'
    ])
    insert_kpis_into_db(session, total_mpf_per_transport_type, TotalMPFPerTransportType, ['TRANSPORT_TYPE', 'Total_MPF'])
    insert_kpis_into_db(session, total_revenue_per_category, TotalRevenuePerCategory, ['CATEGORY', 'SKU_REVENUE'])
    insert_kpis_into_db(session, total_transport_cost_per_consignee, TotalTransportCostPerConsignee, ['CONSIGNEE', 'TRANSPORTATION_COST'])

# Function to insert KPIs into the database
def insert_kpis_into_db(session, df, ModelClass, columns):
    for _, row in df.iterrows():
        # Create a record for each row in the DataFrame and add it to the session
        record = ModelClass(**{col: row[col] for col in columns})
        session.add(record)
    session.commit()
    
    # Try-catch block for error handling during database insertion
    try:
        for _, row in df.iterrows():
            # Insert data into the OptimusData table
            data_record = OptimusData(
                TRANSPORTATION_COST=row.get('TRANSPORTATION_COST', None),
                QTY=row.get('QTY', None),
                SKU_REVENUE=row.get('SKU_REVENUE', None),
                # Map other fields as necessary based on your data structure
            )
            session.add(data_record)
        session.commit()
        print("Data successfully inserted into the database.")
    except Exception as e:
        # Roll back the session in case of an error
        session.rollback()
        print(f"Error inserting data into database: {e}")


C:\Users\TalatZakirhusenSayye\AppData\Local\Temp\ipykernel_7268\1041193305.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
import os

if __name__ == "__main__":
    print("Current working directory:", os.getcwd())
    database_path = 'sqlite:///optimus_data.db'
    print(f"Database path: {database_path}")
    
    engine = create_engine(database_path)
    Base.metadata.create_all(engine)  
    Session = sessionmaker(bind=engine)
    session = Session()
     # Update this path
    file_path = 'C:\\Users\\TalatZakirhusenSayye\\Downloads\\ALPHA.xlsx' 
    quality_report_path = 'C:\\Users\\TalatZakirhusenSayye\\Downloads\\data_quality_report.xlsx'
    process_optimus_data(file_path, quality_report_path)


Current working directory: c:\Users\TalatZakirhusenSayye\Desktop\alpha
Database path: sqlite:///optimus_data.db
Data quality report saved to C:\Users\TalatZakirhusenSayye\Downloads\data_quality_report.xlsx
Shape of the DataFrame before imputation: (112900, 21)
Numerical columns selected for imputation: Index(['Unnamed: 0', 'QTY', 'SKU_REVENUE', 'TRANSPORTATION_COST'], dtype='object')
Shape of imputed data: (112900, 4)
Data successfully inserted into the database.
Data successfully inserted into the database.
Data successfully inserted into the database.
Data successfully inserted into the database.
